<a href="https://www.kaggle.com/code/emigiupponi/crypto-capitalization-apis?scriptVersionId=143449644" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Crypto Capitalization (Coinmetrics API)

# Setup

In [ ]:
!pip install kaleido
!pip install plotly>=4.0.0

In [ ]:
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca

In [ ]:
!chmod +x /usr/local/bin/orca

In [ ]:
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
pip install coinmetrics-api-client

In [ ]:
pip install coinmetrics-api-client -U

In [ ]:
from os import environ
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as image
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
import plotly.express as px
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
%matplotlib inline

In [ ]:
# for plotting
# sns.set_theme()
# sns.set(rc={'figure.figsize':(12,8)})

In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [ ]:
# We recommend privately storing your API key in your local environment.
try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")

# Initialize the client
client = CoinMetricsClient(api_key)

In [ ]:
# Get all assets that have a reference rate 
assets_refrate = client.catalog_metrics("CapMrktCurUSD")
# Get list of assets with daily ref rate 
# uncomment the top line to look at *every* asset with reference rates
# asset_with_ref_rates = assets_refrate[0]["frequencies"][0]["assets"]
asset_with_ref_rates = ['btc', 'eth', 'usdc']
#Query API for prices, daily CM reference rates as dataframe
metrics = "CapMrktCurUSD"
frequency = "1d"
start_time = "2011-11-08"
end_time = datetime.now().strftime('%Y-%m-%d')

logging.info("Getting prices...")
df_CapMrkt = client.get_asset_metrics(
    assets=asset_with_ref_rates,
    metrics=metrics,
    frequency=frequency,
    start_time=start_time,
    end_time=end_time
).to_dataframe()
# Assign datatypes
df_CapMrkt["time"] = pd.to_datetime(df_CapMrkt.time)
df_CapMrkt["CapMrktCurUSD"] = df_CapMrkt.CapMrktCurUSD.astype(float)

# Reshape dataset so assets are in columns, dates are the rows, and the values are prices
df_CapMrkt_pivot = df_CapMrkt.pivot(
    index="time",
    columns="asset",
    values="CapMrktCurUSD"
)

# Index each asset's time series to 1 
for col in df_CapMrkt_pivot.columns:
    logging.info(f"Calculating Reference rate for {col}....")
    # First price in time series
    first_price = df_CapMrkt_pivot[df_CapMrkt_pivot[col].notnull()][col].iloc[0]
    # Index time series
    df_CapMrkt_pivot[col] = df_CapMrkt_pivot[col]/first_price
    # Fill forward for Null values
    df_CapMrkt_pivot[col] = df_CapMrkt_pivot[col].ffill()

In [ ]:
from matplotlib.pyplot import plot

In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
import plotly.graph_objects as go

In [ ]:
import plotly.graph_objects as go
import requests
import tempfile
import matplotlib.pyplot as plt
import pandas as pd

# Descargar la fuente Roboto desde la web
font_url = 'https://github.com/google/fonts/blob/main/apache/roboto/static/Roboto-Regular.ttf?raw=true'
font_data = requests.get(font_url).content

# Guardar el contenido de la fuente en un archivo temporal
with tempfile.NamedTemporaryFile(suffix='.ttf', delete=False) as tmp_file:
    tmp_file.write(font_data)
    font_path = tmp_file.name

# Registrar la fuente Roboto en Matplotlib
plt.rcParams['font.family'] = 'Roboto'

# Filtrar los datos para los últimos 5 años
current_year = pd.Timestamp.now().year
five_years_ago = current_year - 5
df_filtered = df_CapMrkt[df_CapMrkt['time'].dt.year >= five_years_ago]

fig = go.Figure()

# Añadir las áreas correspondientes
for asset, color in zip(df_filtered['asset'].unique(), ['#AA322F', '#1C598A', '#6CADE1']):
    data = df_filtered[df_filtered['asset'] == asset]
    fig.add_trace(go.Scatter(x=data['time'], y=data['CapMrktCurUSD'], fill='tozeroy', mode='lines', name=asset, line=dict(color=color)))

# Personalizar el diseño
fig.update_layout(
    title_text='<b>Crypto Capitalization</b>',  # Título en negrita
    title_font_color="#AA322F",
    yaxis_title='USD',  # Título del eje y
    font=dict(size=20, color="Black"),  # Tamaño de fuente y color
    legend=dict(title_font=dict(color='darkgray'))  # Color de la leyenda en gris oscuro
    #template="plotly_white"  # Plantilla
)

fig.show()


# Online

https://towardsdatascience.com/embedding-interactive-python-plots-on-the-web-84ceab57e417

In [ ]:
pip install chart_studio

In [ ]:
import chart_studio
import plotly.express as px
import chart_studio.plotly as py

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("CHARTSTUDIO_API_KEY")

In [ ]:
chart_studio.tools.set_credentials_file(username='emigiupponi', 
                                        api_key=secret_value_0)

In [ ]:
py.plot(fig, filename = 'cap', auto_open=True)

In [ ]:
secret_value_1 = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")

In [ ]:
chart_studio.tools.set_credentials_file(username='crypto-financial-stability-monitor', 
                                        api_key=secret_value_1)

In [ ]:
py.plot(fig, filename = 'cap', auto_open=True)